In [1]:
import pandas as pd
import sqlite3
from os import path
import json
from time import sleep
from lxml import html
import requests
import os
import datetime
from IPython.display import clear_output

bdPath = path.join("bd.db")
nicksPath = path.join("..", "baseDeNicks.csv")

jogadoresPath = path.join("jogadores.sql")
campeoesJogadorPath = path.join("campeoesJogador.sql")
partidasPath = path.join("partidas.sql")
jogadoresPartidaPath = path.join("jogadoresPartida.sql")
logPath = path.join("nicksAdicionados.log")

In [2]:
bd = sqlite3.connect(bdPath) 

#Adiciona Elos
#elos = open('elos.sql', 'r').read()
#bd.executescript(elos)

#Adiciona Campeoes
#Criando inserts
#campeoesJson = json.load(open('lol-champions.json', 'rb'))
#n = len(campeoesJson)-1
#while (n>-1):
#    print("INSERT INTO campeao('campeaoId', 'nome') VALUES("+campeoesJson[n]['key']+", \""+campeoesJson[n]['name']+"\""+");") 
#    n=n-1
#campeoes = open('campeoes.sql', 'r').read()
#bd.executescript(campeoes)

#elo = pd.read_sql('SELECT * FROM elo', bd)
#campeoes = pd.read_sql('SELECT * FROM campeao', bd)

bd.close()

In [11]:
#Obtem informações do jogador e gera inserts pro bd
def getJogadorData(nick):
    baseUrl = "http://br.op.gg"
    playerUrl = "/summoner/userName="

    urlMontada = baseUrl+playerUrl+nick
    request = requests.get(urlMontada)
    while request.status_code!=200:
        request = requests.get(urlMontada)
    html2 = html.fromstring(request.content)
    getData = html2.xpath('//div[@class="GameListContainer"]')
    if getData==[]:
        return -1
    summonerId = getData[0].attrib['data-summoner-id']

    elo = str(html2.xpath('//div/span[@class="tierRank"]/text()')).replace('[', '').replace(']', '').replace('\'', '')
    if elo=="":
        elo = "Unranked"
    pdl = str(html2.xpath('//div/span[@class="LeaguePoints"]/text()')).replace('[', '').replace(']', '').replace('\\n', '').replace('\\t', '').replace('\'', '').replace(' LP', '').replace(',', '')
    vitorias = str(html2.xpath('//span/span[@class="wins"]/text()')).replace('[', '').replace(']', '').replace('W', '').replace('L', '').replace('\'', '').replace(' LP', '').replace(',', '')
    derrotas = str(html2.xpath('//span/span[@class="losses"]/text()')).replace('[', '').replace(']', '').replace('W', '').replace('L', '').replace('\'', '').replace(' LP', '').replace(',', '')
    winrate = str(html2.xpath('//span/span[@class="winratio"]/text()')).replace('[', '').replace(']', '').replace('%', '').replace('Win Ratio ', '').replace('\'', '').replace(' LP', '')
    
    #TODO: Corrigir pois posições são dinamicamente colocadas após carregamento da página
    #posicoes = str(html2.xpath('//ul[@class="Content"]/li/div[@class="PositionStatContent"]/div/text()')).replace('[', '').replace(']', '').replace('\'', '').replace(' ', '').split(',')
    #posicao1 = posicoes[0]
    #posicao2 = posicoes[1]
    posicao1 = ""
    posicao2 = ""
    
    #DEBUGMODE
    '''
    print("Nick: " + nick + 
          "\nID: " + summonerId + 
          "\nElo: " + elo + 
          " - " + pdl + " PDL" +
          "\nVitórias: " + vitorias + 
          "\nDerrotas: " + derrotas + 
          "\nTaxa de Vitória: " + winrate + "%" +
          "\nPosição 1: " + posicao1 + 
          "\nPosição 2: " + posicao2)
    '''
    
    jogadoresSql = open(jogadoresPath, "a")
    jogadoresSql.write("INSERT INTO jogador('summonerId', 'nick', 'elo', 'pdl', 'vitorias', 'derrotas', 'winrate', 'posicao1', 'posicao2') VALUES("+summonerId+","+"\'"+nick+"\'"+","+"\'"+elo+"\'"+","+pdl+","+vitorias+","+derrotas+","+winrate+","+"\'"+posicao1+"\'"+","+"\'"+posicao2+"\');\n")
    jogadoresSql.close()
    return summonerId 

In [31]:
#Obtem informações dos campeoes do jogador e gera inserts pro bd
def getCampeoesJogadorData(summonerId):
    baseUrl = "http://br.op.gg"
    champsUrl = "/summoner/champions/ajax/champions.rank/summonerId="
    season = "&season=12"

    urlMontada = baseUrl+champsUrl+summonerId+season
    request = requests.get(urlMontada)
    html2 = html.fromstring(request.content)

    nCampeoes = html2.xpath('//tr[@class="Row TopRanker"]')
    for i in range(len(nCampeoes)):
        pref = html2.xpath('//tr[@class="Row TopRanker"]/td[@class="Rank Cell"]/text()')[i]
        nome = html2.xpath('//tr[@class="Row TopRanker"]/td[@class="ChampionName Cell"]/a/text()')[i]
        vitorias = html2.xpath('//tr[@class="Row TopRanker"]/td[@class="RatioGraph Cell"]/div/div/div[@class="Text Left"]/text()')    
        if i>=len(vitorias):
            vitorias = '0'
        else:
            vitorias = vitorias[i].replace("W", "")

        derrotas = html2.xpath('//tr[@class="Row TopRanker"]/td[@class="RatioGraph Cell"]/div/div/div[@class="Text Right"]/text()')
        if i>=len(derrotas):
            derrotas = '0'
        else:
            derrotas = derrotas[i].replace("L", "")
        
        partidas = str(int(vitorias)+int(derrotas))
        winrate = html2.xpath('//tr[@class="Row TopRanker"]/td[@class="RatioGraph Cell"]/div/span/text()')[i].replace("%", "")
        kda = html2.xpath('//tr[@class="Row TopRanker"]/td[contains(@class, \'KDA \')]')[i].attrib['data-value']

        #TODO: Corrigir separação de cada valor pois o site é mal estruturado nesse ponto
        #values = html2.xpath('//tr[@class="Row TopRanker"]/td[@class="Value Cell"]/text()')[i].split(' ')
        #values = str(values).replace('\\n', "").replace('\\t', "").replace('[', "").replace(']', "").replace(',', "").replace('\'', "")
        #values = str(values).splitlines()
        #ouro = values[0]
        #cs = values[1]
        #danoCausado = values[4]
        #danoRecebido = values[5]

        #DEBUGMODE
        '''
        print("ID: " + summonerId + 
              " Campeão: " + nome + 
              " Preferência: " + pref + 
              " Partidas: " + partidas + 
              " Vitórias: " + vitorias + 
              " Derrotas: " + derrotas + 
              " Taxa de Vitória: " + winrate + "%" +
              " KDA: " + kda)
        '''

        #print("\nINSERT INTO campeao_jogador('summonerId', 'campeaoNome', 'campeaoPref', 'partidas', 'vitorias', 'derrotas', 'winrate', 'kda', 'ouro', 'cs', 'danoCausado', 'danoRecebido') VALUES("
             # +summonerId+","+"\'"+nome+"\'"+","+pref+","+vitorias+derrotas+","+vitorias+","+derrotas+","+winrate+","+","+kda+","+ouro+","+cs+","+danoCausado+","+danoRecebido+");")

        campeoesJogadorSql = open(campeoesJogadorPath, "a")
        campeoesJogadorSql.write("INSERT INTO campeao_jogador('summonerId', 'campeaoNome', 'campeaoPref', 'partidas', 'vitorias', 'derrotas', 'winrate', 'kda', 'ouro', 'cs', 'danoCausado', 'danoRecebido') VALUES("
          +summonerId+","+"\""+nome+"\""+","+pref+","+partidas+","+vitorias+","+derrotas+","+winrate+","+kda+","+'0'+","+'0'+","+'0'+","+'0'+");\n")
        campeoesJogadorSql.close()

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
    
#Funcao de apoio - converte nick em id
def findIdByName(summonerNome):
    baseUrl = "http://br.op.gg"
    playerUrl = "/summoner/userName="   

    urlMontada = baseUrl+playerUrl+summonerNome
    request = requests.get(urlMontada)
    html2 = html.fromstring(request.content)
    getData = html2.xpath('//div[@class="GameListContainer"]')
    summonerId = getData[0].attrib['data-summoner-id']

    return summonerId
    
#Obtem informações das partidas do jogador e gera inserts pro bd
def getPartidasJogadorData(summonerId):   
    baseUrl = "http://br.op.gg"
    champsUrl = "/summoner/matches/ajax/averageAndList/startInfo=0&summonerId="
    partidaUrl = "/summoner/matches/ajax/detail/gameId="
    seasonType = "&type=soloranked&season=12"
    
    urlMontada = baseUrl+champsUrl+summonerId+seasonType
    driver = webdriver.Chrome()
    driver.get(urlMontada)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    listaPartidas = soup.findAll("div", {"class": "GameItemWrap"})
    
    import json
    try:
        js = json.loads(soup.text)
    except ValueError:
        return -1
    
    teste = BeautifulSoup(js["html"], 'html.parser')
    listaPartidas = teste.findAll("div", {"class": "GameItemWrap"})

    campeoes = teste.findAll("div", {"class": "ChampionName"})
    kdas = teste.findAll("span", {"class": "KDARatio"})
    kdas.pop(0)
    desc = teste.findAll("td", {"class": "SummonerName"})    
    partidas = teste.findAll("div", {"class": "GameItem"})
    

    for i in range(len(listaPartidas)):
    #for i in range(1):
        sumId = partidas[i].get('data-summoner-id')
        gameTime = partidas[i].get('data-game-time')
        gameId = partidas[i].get('data-game-id')
        
        #Informações da partida
        urlMontada2 = baseUrl+partidaUrl+gameId+"&summonerId="+sumId+"&gameTime="+gameTime
        request = requests.get(urlMontada2)
        html2 = html.fromstring(request.text)
        
        values = ['0','0','0','0','0','0']
        summary = html2.xpath('//div[contains(@class, \'Team\')]/div[@class="ObjectScore"]/text()') 
        for k in summary[1::2]:
            values.append(k.strip())            
        baroesBlue = values[0]
        dragoesBlue = values[1]
        torresBlue = values[2]
        baroesRed = values[3]
        dragoesRed = values[4]
        torresRed = values[5]
        

        dadosBlue = html2.xpath('//div[@class="text graph--data graph--data__left"]/text()')
        dadosRed = html2.xpath('//div[@class="text graph--data graph--data__right"]/text()')        
        
        if len(dadosBlue)>1:
            killsBlue = dadosBlue[0]
            killsRed = dadosRed[0]
            ouroBlue = dadosBlue[1]
            ouroRed = dadosRed[1]
        else:
            killsBlue = '0'
            killsRed = '0'
            ouroBlue = '0'
            ouroRed = '0'           
        
        gameResult = html2.xpath('//th[@class="HeaderCell"]/span[@class="GameResult"]/text()')
        if gameResult[0].replace("\t", "").replace("\n", "") == "Victory":
            if html2.xpath('//tr/th[@class="HeaderCell"]/text()')[1].replace("\t", "").replace("\n", "") == "(Blue Team)":
                vitoria = "0"
            else:
                vitoria = "1"
        if gameResult[0].replace("\t", "").replace("\n", "") == "Defeat":
            if html2.xpath('//tr/th[@class="HeaderCell"]/text()')[1].replace("\t", "").replace("\n", "") == "(Blue Team)":
                vitoria = "1"
            else:
                vitoria = "0"
            
        #DEBUGMODE PARTIDA  
        '''
        print("GameID: " + gameId  + 
              " Vitória: " + vitoria +              
              " BaroesBlue: " + baroesBlue +
              " DragoesBlue: " + dragoesBlue  + 
              " TorresBlue: " + torresBlue  + 
              " BaroesRed: " + baroesRed + 
              " DragoesRed: " + dragoesRed  + 
              " TorresRed: " + torresRed  + 
              " KillsBlue: " + killsBlue  + 
              " KillsRed: " + killsRed  + 
              " OuroBlue: " + ouroBlue  + 
              " OuroRed: " + ouroRed)
        '''

        #print("INSERT INTO partida('gameId', 'vitoria', 'baroesBlue', 'dragoesBlue', 'torresBlue', 'baroesRed', 'dragoesRed', 'torresRed', 'killsBlue', 'killsRed', 'ouroBlue', 'ouroRed') VALUES("
        #      +gameId+","+vitoria+","+baroesBlue+","+dragoesBlue+","+torresBlue+","+baroesRed+","+dragoesRed+","+torresRed+","+killsBlue+","+killsRed+","+ouroBlue+","+ouroRed+");\n")
        partidasSql = open(partidasPath, "a")
        partidasSql.write("INSERT INTO partida('gameId', 'vitoria', 'baroesBlue', 'dragoesBlue', 'torresBlue', 'baroesRed', 'dragoesRed', 'torresRed', 'killsBlue', 'killsRed', 'ouroBlue', 'ouroRed') VALUES("
              +gameId+","+vitoria+","+baroesBlue+","+dragoesBlue+","+torresBlue+","+baroesRed+","+dragoesRed+","+torresRed+","+killsBlue+","+killsRed+","+ouroBlue+","+ouroRed+");\n")
        partidasSql.close()

        
        
        #Jogadores da partida
        html3 = html.fromstring(request.text)
        nicks = html3.xpath('//td[@class="SummonerName Cell"]/a/text()') 
        campeoes = html2.xpath('//tr[contains(@class, \'Row\')]/td/a/div[contains(@class, \'tip\')]/text()')
        kdas = html2.xpath('//td[@class="KDA Cell"]/span[contains(@class, \'KDARatio\')]/text()') #.replace(":1", "")
        danos = html2.xpath('//div[@class="ChampionDamage"]/text()') #replace(",", "")
        wards = html2.xpath('//span[@class="SightWard"]/text()')
        css = html2.xpath('//div[@class="CS"]/text()')
        
        #DEBUGMODE JOGADORES DA PARTIDA  
        '''
        for i in range(10):
            print("GameID: " + gameId  + 
              " SummonerId: " + nicks[i] +              
              " Campeão: " + campeoes[i] +
              " KDA: " + kdas[i].replace(":1", "")  + 
              " Dano: " + danos[i].replace(",", "")  + 
              " Wards: " + wards[i] + 
              " CS: " + css[i])
        '''
        
        for i in range(10):
            summonerId = findIdByName(nicks[i])
            campeaoNome = campeoes[i]
            kdaPartida = kdas[i].replace(":1", "").replace("Perfect", "10.00")
            danoPartida = danos[i].replace(",", "")
            wardsPartida = wards[i]
            csPartida = css[i]
            #print("INSERT INTO jogador_partida('gameId', 'summonerId', 'campeaoNome', 'kdaPartida', 'danoPartida', 'wardsPartida', 'csPartida') VALUES("
            #  +gameId+","+summonerId+",\""+campeaoNome+"\","+kdaPartida+","+danoPartida+","+wardsPartida+","+csPartida+");\n")
            #'''
            jogadoresPartidaSql = open(jogadoresPartidaPath, "a")
            jogadoresPartidaSql.write("INSERT INTO jogador_partida('gameId', 'summonerId', 'campeaoNome', 'kdaPartida', 'danoPartida', 'wardsPartida', 'csPartida') VALUES("
              +gameId+","+summonerId+",\""+campeaoNome+"\","+kdaPartida+","+danoPartida+","+wardsPartida+","+csPartida+");\n")
            jogadoresPartidaSql.close()
            #'''

In [10]:
def getChampionName(championId):
    bd = sqlite3.connect(bdPath) 
    cursor = bd.cursor()
    cursor.execute('SELECT (campeao.nome) FROM campeao WHERE campeaoId='+championId)
    for linha in cursor.fetchall():
        champ = linha[0]
        break
    bd.close()
    return champ

def getSummonerId(nick):
    bd = sqlite3.connect(bdPath) 
    sId = -1
    cursor = bd.cursor()
    cursor.execute('SELECT (jogador.summonerId) FROM jogador WHERE nick=\''+nick+'\'')
    for linha in cursor.fetchall():
        sId = linha[0]
        break
    bd.close()
    return sId

#Obtem informações das partidas do jogador e gera inserts pro bd a partir da api
def getPartidaData(gameId):
    API_KEY = 'RGAPI-1878d2cd-aa9f-4414-8051-f8139d97699f'
    REGIAO = 'br1'
    URL_BASE = 'https://'+REGIAO+'.api.riotgames.com/'

    urlMontada = URL_BASE+'lol/match/v4/matches/'+gameId+'?api_key='+API_KEY
    response = requests.get(urlMontada)
    print(urlMontada, response.status_code)
    while response.status_code != 200:
        sleep(3)
        response = requests.get(urlMontada)
        

    jsonPartida = response.json()

    firstBlood=firstTower=firstInhibitor=firstBaron=firstDragon=firstRiftHerald=vitoria='-1'

    if jsonPartida['teams'][0]['teamId']==100:
        timeBlue = jsonPartida['teams'][0]  
    else:
        timeRed = jsonPartida['teams'][0] 

    if jsonPartida['teams'][1]['teamId']==200:
        timeRed = jsonPartida['teams'][1]  
    else:
        timeBlue = jsonPartida['teams'][1]     

    if timeBlue['win']=='Win':
        vitoria = '0'
    if timeBlue['firstBlood']:
        firstBlood = '0'
    if timeBlue['firstTower']:
        firstTower = '0'
    if timeBlue['firstInhibitor']:
        firstInhibitor = '0'
    if timeBlue['firstBaron']:
        firstBaron = '0'
    if timeBlue['firstDragon']:
        firstDragon = '0'
    if timeBlue['firstRiftHerald']:
        firstRiftHerald = '0'        
    if timeRed['win']=='Win':
        vitoria = '1'
    if timeRed['firstBlood']:
        firstBlood = '1'
    if timeRed['firstTower']:
        firstTower = '1'
    if timeRed['firstInhibitor']:
        firstInhibitor = '1'
    if timeRed['firstBaron']:
        firstBaron = '1'
    if timeRed['firstDragon']:
        firstDragon = '1'
    if timeRed['firstRiftHerald']:
        firstRiftHerald = '1'

    t1_towerKills = str(timeBlue['towerKills'])
    t1_inhibitorKills = str(timeBlue['inhibitorKills'])
    t1_baronKills = str(timeBlue['baronKills'])
    t1_dragonKills = str(timeBlue['dragonKills'])
    t1_riftHeraldKills = str(timeBlue['riftHeraldKills'])
    t2_towerKills = str(timeRed['towerKills'])
    t2_inhibitorKills = str(timeRed['inhibitorKills'])
    t2_baronKills = str(timeRed['baronKills'])
    t2_dragonKills = str(timeRed['dragonKills'])
    t2_riftHeraldKills = str(timeRed['riftHeraldKills'])

    killsBlue = 0
    goldBlue = 0
    killsRed = 0
    goldRed = 0
    for p in jsonPartida['participants']:
        if p['teamId']==100:
            killsBlue += p['stats']['kills']
            goldBlue += p['stats']['goldEarned']
        else:
            killsRed += p['stats']['kills']
            goldRed += p['stats']['goldEarned']

    killsBlue = str(killsBlue)
    goldBlue = str(goldBlue)
    killsRed = str(killsRed)
    goldRed = str(goldRed)
    
    #DEBUG
    #print("INSERT INTO partida('gameId', 'vitoria', 'firstBlood', 'firstTower', 'firstInhibitor', 'firstBaron','firstDragon', 'firstRiftHerald', 't2_baronKills', 't1_baronKills', 't2_dragonKills', 't1_dragonKills', 't2_riftHeraldKills', 't1_riftHeraldKills', 't2_towerKills', 't1_towerKills', 't2_inhibitorKills', 't1_inhibitorKills','killsRed', 'killsBlue', 'goldRed', 'goldBlue') VALUES("
 #+gameId+","+vitoria+","+firstBlood+","+firstTower+","+firstInhibitor+","+firstBaron+","+firstDragon+","+firstRiftHerald+","+t2_baronKills+","+t1_baronKills+","+t2_dragonKills+","+t1_dragonKills+","+t2_riftHeraldKills+","+t1_riftHeraldKills+","+t2_towerKills+","+t1_towerKills+","+t2_inhibitorKills+","+t1_inhibitorKills+","+killsRed+","+killsBlue+","+goldRed+","+goldBlue+");\n")
    partidasSql = open(partidasPath, "a")
    partidasSql.write("INSERT INTO partida('gameId', 'vitoria', 'firstBlood', 'firstTower', 'firstInhibitor', 'firstBaron','firstDragon', 'firstRiftHerald', 't2_baronKills', 't1_baronKills', 't2_dragonKills', 't1_dragonKills', 't2_riftHeraldKills', 't1_riftHeraldKills', 't2_towerKills', 't1_towerKills', 't2_inhibitorKills', 't1_inhibitorKills','killsRed', 'killsBlue', 'goldRed', 'goldBlue') VALUES("
    +gameId+","+vitoria+","+firstBlood+","+firstTower+","+firstInhibitor+","+firstBaron+","+firstDragon+","+firstRiftHerald+","+t2_baronKills+","+t1_baronKills+","+t2_dragonKills+","+t1_dragonKills+","+t2_riftHeraldKills+","+t1_riftHeraldKills+","+t2_towerKills+","+t1_towerKills+","+t2_inhibitorKills+","+t1_inhibitorKills+","+killsRed+","+killsBlue+","+goldRed+","+goldBlue+");\n")
    partidasSql.close()

    #DEBUG
    '''
    print("GameID: " + gameId  + 
          "\nVitória: " + vitoria +              
          "\nBaroesBlue: " + t1_baronKills +
          "\nDragoesBlue: " + t1_dragonKills  + 
          "\nTorresBlue: " + t1_towerKills  + 
          "\nInibidoresBlue: " + t1_inhibitorKills  + 
          "\nArautosBlue: " + t1_riftHeraldKills  + 
          "\nBaroesRed: " + t2_baronKills + 
          "\nDragoesRed: " + t2_dragonKills  + 
          "\nTorresRed: " + t2_towerKills  + 
          "\nInibidoresRed: " + t2_inhibitorKills  + 
          "\nArautosRed: " + t2_riftHeraldKills  + 
          "\nKillsBlue: " + killsBlue  + 
          "\nKillsRed: " + killsRed  + 
          "\nOuroBlue: " + goldBlue  + 
          "\nOuroRed: " + goldRed)
    '''
    
    playerNames = ['','','','','','','','','','','']
    for p in jsonPartida['participantIdentities']:
        playerNames[p['participantId']] = p['player']['summonerName']
    playerNames.pop(0)

    for p in jsonPartida['participants']:
        pId = p['participantId']
        campeaoNome = str(getChampionName(str(p['championId'])))
        summonerId = str(getSummonerId(playerNames[pId-1].strip()))
        time = str(int((p['teamId']/100)-1))
        kdaPartida = str(round((p['stats']['kills']+p['stats']['assists'])/max(p['stats']['deaths'],1),2))
        danoPartida = str(p['stats']['totalDamageDealtToChampions'])
        wardsPartida = str(p['stats']['wardsPlaced'])
        csPartida = str(p['stats']['totalMinionsKilled']+p['stats']['neutralMinionsKilled'])
        
        #DEBUG
        #print("INSERT INTO jogador_partida('gameId', 'time', 'summonerId', 'campeaoNome', 'kdaPartida', 'danoPartida', 'wardsPartida', 'csPartida') VALUES("
         #    +gameId+","+time+","+summonerId+",\""+campeaoNome+"\","+kdaPartida+","+danoPartida+","+wardsPartida+","+csPartida+");\n")
        jogadoresPartidaSql = open(jogadoresPartidaPath, "a")
        jogadoresPartidaSql.write("INSERT INTO jogador_partida('gameId', 'time', 'summonerId', 'campeaoNome', 'kdaPartida', 'danoPartida', 'wardsPartida', 'csPartida') VALUES("
         +gameId+","+time+","+summonerId+",\""+campeaoNome+"\","+kdaPartida+","+danoPartida+","+wardsPartida+","+csPartida+");\n")
        jogadoresPartidaSql.close()

In [ ]:
partidasList = open("partidasOld.sql", "r")
partidas = [line.rstrip('\n')[179:189] for line in partidasList]
for p in partidas:
    getPartidaData(str(p))
    sleep(1.3)

In [ ]:
#Apagar Arquivos de Teste
'''
if(os.path.isfile(logPath)):
    os.remove(logPath)
if(os.path.isfile(jogadoresPath)):
    os.remove(jogadoresPath)
if(os.path.isfile(campeoesJogadorPath)):
    os.remove(campeoesJogadorPath)
if(os.path.isfile(partidasPath)):
    os.remove(partidasPath)
if(os.path.isfile(jogadoresPartidaPath)):
    os.remove(jogadoresPartidaPath)
'''

In [110]:
nick = pd.read_csv(nicksPath)
nick.drop_duplicates(subset ="nick", 
                     keep = False, inplace = True)
nicks = nick['nick']

nicks2 = nicks[56350:]

In [ ]:
#Jogadores coletados: 66678
for jogador in nicks2:
    log = open(logPath, "a")
    logS = str(datetime.datetime.now())+" - "+jogador+"\n"
    print(logS, end='\r\r')
    log.write(logS)
    log.close()
    summonerId = getJogadorData(jogador.strip())  

In [ ]:
#Captura dos campeoes da base de jogadores - sucesso
idss = ids['summonerId']
for idd in idss:
    logS = str(datetime.datetime.now())+" - "+str(idd)+"\n"
    print(logS, end='\r\r')
    getCampeoesJogadorData(str(idd))

In [ ]:
#Captura das partidas da base de jogadores - sucesso
idss = ids['summonerId']
idss2 = idss[3291:]
idss2.iloc[0]
for idd in idss2:  
    logS = str(datetime.datetime.now())+" - "+str(idd)+"\n"
    print(logS, end='\r\r')
    getPartidasJogadorData(str(idd))

In [ ]:
idss2 = idss[3291:]
idss2.iloc[0]

for n in range(len(idss)):  
    logS = str(datetime.datetime.now())+" - "+str(idss[n])+"\n"
    if idss[n]==840919:
        print(n)
        break

In [13]:
#Executa scrips de insert 
bd = sqlite3.connect(bdPath) 
jogador = open(jogadoresPath, 'r').read()
#bd.executescript(jogador)

campeoesJogador = open(campeoesJogadorPath, 'r').read()
#bd.executescript(campeoesJogador)

partida = open(partidasPath, 'r').read()
bd.executescript(partida)

jogadoresPartida = open(jogadoresPartidaPath, 'r').read()
bd.executescript(jogadoresPartida)

#ids = pd.read_sql('SELECT (jogador.summonerId) FROM jogador', bd)
bd.close()